# Data Structures

To keep this documentation generic we typically use dimensions `x` or `y`, but this should *not* be seen as a recommendation to use these labels for anything but actual positions or offsets in space.

## Variable

### Basics

[scipp.Variable](../../generated/classes/scipp.Variable.rst#scipp.Variable) is a labeled multi-dimensional array.
A variable has the following key properties:

- `values`: a multi-dimensional array of values, e.g., similar to a [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray)
- `variances`: a (optional) multi-dimensional array of variances for the array values
- `dims`: a list of dimension labels (strings) for each axis of the array
- `unit`: a (optional) physical unit of the values in the array

Note that variables, unlike [DataArray](data-structures.ipynb#DataArray) and its eponym [xarray.DataArray](https://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray), do *not* have coordinate dicts.

In [ ]:
import numpy as np
import scipp as sc

Variables should generally be created using one of the available [creation functions](../../reference/creation-functions.rst#creation-functions).
For example, we can create a variable from a NumPy array:

In [ ]:
var = sc.array(dims=['x', 'y'], values=np.random.rand(2, 4), unit='s')

Using a unit is optional but highly encouraged if the variable represents a physical quantity.
See [Creating Arrays and Datasets](./creating-arrays-and-datasets.rst) for an overview of the different methods for creating variables.

<div class="alert alert-info">

**Note:**

Internally Scipp does not use NumPy, so the above makes a *copy* of the numpy array of values into an internal buffer.
    
</div>

We can inspect the created variable as follows:

In [ ]:
sc.show(var)

In [ ]:
var

<div class="alert alert-warning">
    <b>WARNING:</b>

The above makes use of IPython's rich output representation, but relying on this feature has a common pitfall:
    
IPython (and thus Jupyter) has an [Output caching system](https://ipython.readthedocs.io/en/stable/interactive/reference.html?highlight=previous#output-caching-system).
By default this keeps the last 1000 cell outputs.
In the above case this is `var` (not the displayed HTML, but the object itself).
If such cell outputs are large then this output cache can consume enormous amounts of memory.

Note that `del var` will *not* release the memory, since the IPython output cache still holds a reference to the same object.
See [this FAQ entry](../../getting-started/faq.rst#scipp-is-using-more-and-more-memory-the-jupyter-kernel-crashes) for clearing or disabling this caching.

</div>

In [ ]:
var.unit

In [ ]:
var.values

### 0-D variables (scalars)

A 0-dimensional variable contains a single value (and an optional variance).

In [ ]:
scalar = sc.scalar(1.2, unit='s')
sc.show(scalar)
scalar

Singular versions of the `values` and `variances` properties are provided:

In [ ]:
print(scalar.value)
print(scalar.variance)

An exception is raised from the `value` and `variance` properties if the variable is not 0-dimensional.

<div class="alert alert-info">

**Note:**

Scalar variables are distinct from arrays that contain a single value.
For example, `sc.scalar(1)` is equivalent to `sc.array(dims=[], values=1)`.
But all the following are distinct:

 - `sc.array(dims=[], values=1)`
 - `sc.array(dims=['x'], values=[1])`
 - `sc.array(dims=['x', 'y'], values=[[1]])`

In particular, the first is a scalar while the other two are not; they are arrays with an extent of one.
Accessing the `value` property of one of the latter two variables would raise an exception because this property requires a 0-dimensional variable.

</div>

## DataArray

### Basics

[scipp.DataArray](../../generated/classes/scipp.DataArray.rst#scipp.DataArray) is a labeled array with associated coordinates.
A data array is essentially a [Variable](../../generated/classes/scipp.Variable.rst#scipp.Variable) object with attached dicts of coordinates and masks.

A data array has the following key properties:

- `data`: the variable holding the array data (its values, variances, dims, and unit).
- `coords`: a dict-like container of coordinates for the array.
- `masks`: a dict-like container of masks for the array.

All dict-likes are accessed using a string as key.

`coords` can be seen as independent variables of the data array, while `data` is the dependent variable.
This means that coordinates must generally match in operations between data arrays.
See the section on [alignment](../computation.ipynb#Alignment) in the computation guide for details.

`masks` allows for storing boolean-valued masks alongside data.

`data` as well as the individual values of the `coords` and `masks` dictionaries are of type [Variable](../../generated/classes/scipp.Variable.rst#scipp.Variable), i.e., they have a physical unit and can be used for computation.

<div class="alert alert-warning">

    **Deprecation**

Data arrays also have `attrs` which are essentially unaligned coordinates.
`attrs` have been deprecated in version 23.09.0 and will be removed in version 24.12.0.
See [ADR 0016](../../development/adr/0016-do-not-support-attrs.rst)
</div>

A data array can be created from variables for its constituents as follows:

In [ ]:
da = sc.DataArray(
    data=sc.array(dims=['y', 'x'], values=np.random.rand(2, 3)),
    coords={
        'y': sc.array(dims=['y'], values=np.arange(2.0), unit='m'),
        'x': sc.array(dims=['x'], values=np.arange(3.0), unit='m'),
    },
    masks={
        'm': sc.array(dims=['x'], values=[False, True, False]),
    },
)
sc.show(da)

The dict-like `coords` and `masks` properties give access to the respective underlying variables:

In [ ]:
da.coords['x']

In [ ]:
da.masks['m']

Unlike `values` when creating a variable, `data` as well as entries in the metadata dicts (`coords` and `masks`) are *not* deep-copied on insertion into a data array.
To avoid unwanted sharing, call the `copy()` method.
Compare:

In [ ]:
x2 = sc.zeros(dims=['x'], shape=[3])
da.coords['x2_shared'] = x2
da.coords['x2_copied'] = x2.copy()
x2 += 123
da

Meta data can be removed in the same way as in Python dicts:

In [ ]:
del da.coords['x2_shared']

Alignment of coordinates can be queried with the `aligned` property:

In [ ]:
da.coords['x'].aligned

It can be set using the `set_aligned` method of the coordinates:

In [ ]:
da.coords.set_aligned('x', False)
da.coords['x'].aligned

Note that the alignment is encoded in [Variable](../../generated/classes/scipp.Variable.rst#scipp.Variable).
It is, however, only meaningful in a `coords` dict.
Scipp ignores alignment in operations of plain variables and when handling masks.

The alignment flag is preserved when inserting variables into `coords` dicts:

In [ ]:
da2 = sc.DataArray(sc.arange('x', 3), coords={'x': da.coords['x']})
da2.coords['x'].aligned

## Dataset

[scipp.Dataset](../../generated/classes/scipp.Dataset.rst#scipp.Dataset) is a dict-like container of data arrays.
Individual items of a dataset ("data arrays") are accessed using a string as a dict key.

In a dataset the coordinates of the sub-arrays are enforced to be *aligned*.
That is, a dataset is not simply a dict of data arrays.
Instead, the individual arrays share their coordinates.
It is therefore not possible to combine *arbitrary* data arrays into a dataset.
If, e.g., the extents in a certain dimension mismatch, or if coordinate values mismatch, insertion of the mismatching data array will fail.

Often a dataset is not created from individual data arrays.
Instead we may provide a dict of variables (the data of the items), and dicts for coords:

In [ ]:
ds = sc.Dataset(
    data={
        'a': sc.array(dims=['y', 'x'], values=np.random.rand(2, 3)),
        'b': sc.array(dims=['x', 'y'], values=np.random.rand(3, 2)),
    },
    coords={
        'x': sc.array(dims=['x'], values=np.arange(3.0), unit='m'),
        'y': sc.array(dims=['y'], values=np.arange(2.0), unit='m'),
        'aux': sc.array(dims=['x'], values=np.random.rand(3)),
    },
)
sc.show(ds)

In [ ]:
ds

In [ ]:
ds.coords['x'].values

The name of a data item serves as a dict key.
Item access returns a new data array which is a view onto the data in the dataset and its corresponding coordinates, i.e., no deep copy is made:

In [ ]:
sc.show(ds['a'])
ds['a']

Use the `copy()` method to turn the view into an independent object:

In [ ]:
copy_of_a = ds['a'].copy()
copy_of_a += 17  # does not change d['a']
ds

Each data item is linked to its corresponding coordinates and masks.
These are accessed using the `coords` and `masks` properties.
The variable holding the data of the dataset item is accessible via the `data` property:

In [ ]:
ds['a'].data

For convenience, properties of the data variable are also properties of the data item:

In [ ]:
ds['a'].values

In [ ]:
ds['a'].variances

In [ ]:
ds['a'].unit

All variables in a dataset must have consistent dimensions.
Thanks to labeled dimensions, transposed data is supported.
Compare items `'a'` and `'b'`:

In [ ]:
ds['a']

In [ ]:
ds['b']

When inserting a data array or variable into a dataset ownership is shared by default.
Use the `copy()` method to avoid this if undesirable:

In [ ]:
ds['da_shared'] = da
ds['da_copied'] = da.copy()
da += 1000
ds

The usual `dict`-like methods are available for `Dataset`:

In [ ]:
for name in ds:
    print(name)

In [ ]:
'a' in ds

In [ ]:
'e' in ds

## DataGroup

[scipp.DataGroup](../../generated/classes/scipp.DataGroup.rst#scipp.DataGroup) is a dict-like container for arbitrary Scipp or Python objects.
Unlike [Dataset](#Dataset), `DataGroup` does not have coords and does not enforce compatible dimensions of its items.
A `DataGroup` can contain other `DataGroup` objects and thus allows for representing tree-like data.
It can be created like a Python `dict`:

In [ ]:
import numpy as np

import scipp as sc

dg = sc.DataGroup(
    a=sc.arange('x', 4),
    b=sc.arange('x', 6),
    c=sc.arange('y', 2),
    d=np.ones((2, 3)),
    e='a string',
)
dg

Just like [DataArray](#DataArray), `DataGroup` provides properties such as `dims`, `shape`, and `sizes`:

In [ ]:
dg.dims

In [ ]:
dg.shape

In [ ]:
dg.sizes

The properties return the union of these properties over all the items in the data group.
Non-Scipp objects are considered to have `dims=()` and `shape=()`.
When items have inconsistent size along a dimension then `shape` and `sizes` report this as `None`.

`DataGroup` supports [positional indexing](../slicing.ipynb#Positional-indexing) if the shape along the indexed dimension is unique.
[Label-based indexing](../slicing.ipynb#Label-based-indexing) is supported if all items have a corresponding coordinate, even if the shape is not unique.

Most Scipp operations also work for `DataGroup`, provided that the operation works for all items in the group.
That is, operations will generally fail if the data group contains non-Scipp objects such as NumPy arrays or other Python objects such as integers or strings.